# 3 - Selezione dei clienti target

Dal dataset che rappresenta i valori RFM per i Best Customers costruisco il dataset completo di attributi inerente ai clienti selezionati. Il nostro target è rappresentato dai clienti con un alto valore di monetary ed una elevata propensione ad acquistare grandi stock di prodotti a prezzi scontati. Al fine di ricavare la percentuale di sconto per l'acquisto di un prodotto utilizzo la differenza in percentuale rispetto alla media (il valore quindi sarà positivo per acquisti di un prodotto ad un prezzo inferiore alla media, negativo altrimenti).

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
best_cust = pd.read_csv('best_customers.csv')

In [3]:
best_cust.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550 entries, 0 to 549
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   CustomerID  550 non-null    float64
 1   Recency     550 non-null    float64
 2   Frequency   550 non-null    float64
 3   Monetary    550 non-null    float64
 4   ClusterID   550 non-null    int64  
dtypes: float64(4), int64(1)
memory usage: 21.6 KB


Seleziono, per la seguente analisi, esclusivamente il sottoinsieme dei primi 200 Best Customers, ordinati per l'attributo Monetary. Tali clienti possono anche essere definiti ***Whales***

In [4]:
best_cust = best_cust.sort_values(by=['Monetary', 'Recency'],
                                  ascending=(False, True)).head(200)

In [5]:
best_cust.head()

,CustomerID,Recency,Frequency,Monetary,ClusterID
159,14309.0,9.0,9.0,228.00,3
344,16126.0,30.0,4.0,227.40,3
322,15861.0,13.0,8.0,227.27,3
485,17567.0,8.0,8.0,226.94,3
220,14776.0,52.0,11.0,226.38,3


In [6]:
best_cust.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200 entries, 159 to 70
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   CustomerID  200 non-null    float64
 1   Recency     200 non-null    float64
 2   Frequency   200 non-null    float64
 3   Monetary    200 non-null    float64
 4   ClusterID   200 non-null    int64  
dtypes: float64(4), int64(1)
memory usage: 9.4 KB


# Innanzitutto individuo i clienti con una più alta tendenza ad acquistare grandi quantità di prodotti a prezzi scontati, preparo quindi il dataset per tale tipo di analisi

## 1) Individuo il set di prodotti interessati e snellisco il dataset

In [11]:
#per calcolare il prezzo medio utilizzo il dataset contenente 
#tutte le transazioni
df = pd.read_csv("cleaned_online_retail.csv")

Al fine di snellire il dataset, escludo i prodotti che non sono mai stati acquistati da nessun best customer

In [12]:
stock_codes_of_target_customers = df[df['CustomerID'].isin(
    list(best_cust['CustomerID'].unique()))]['StockCode'].unique()

In [13]:
print(len(stock_codes_of_target_customers))
print(len(df['StockCode'].unique()))
stock_codes_of_target_customers

2410
3664


array(['84879', '22745', '22748', ..., '22069', '21109', '21111'],
      dtype=object)

In [14]:
cut_df = df[df['StockCode'].isin(stock_codes_of_target_customers)]
cut_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 378991 entries, 0 to 397880
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    378991 non-null  int64  
 1   StockCode    378991 non-null  object 
 2   Description  378991 non-null  object 
 3   Quantity     378991 non-null  int64  
 4   InvoiceDate  378991 non-null  object 
 5   UnitPrice    378991 non-null  float64
 6   CustomerID   378991 non-null  float64
 7   Country      378991 non-null  object 
 8   Sales        378991 non-null  float64
dtypes: float64(3), int64(2), object(4)
memory usage: 28.9+ MB


## 2) Calcolo la media del prezzo di vendita per ogni singolo prodotto

In [15]:
cut_df['PriceMean'] = 0

/Users/leoar/Desktop/DATA_MINING_PROJ/dev/venv/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [16]:
for item in list(cut_df['StockCode'].unique()):
    #df.loc[df[<some_column_name>] == <condition>, [<another_column_name>]]
    cut_df.loc[cut_df['StockCode']==item, ['PriceMean']] = 
        (cut_df[cut_df['StockCode']==item]['UnitPrice'].mean())

/Users/leoar/Desktop/DATA_MINING_PROJ/dev/venv/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [17]:
cut_df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Sales,PriceMean
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30,2.893106
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,3.768403
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00,3.816311
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,4.001595
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,4.054756


In [18]:
cut_df[cut_df['StockCode']=='85123A'].describe()

,InvoiceNo,Quantity,UnitPrice,CustomerID,Sales,PriceMean
count,2035.000000,2035.000000,2035.000000,2035.000000,2035.000000,2.035000e+03
mean,556751.394595,18.074693,2.893106,15561.569042,49.436609,2.893106e+00
std,13055.837231,76.000338,0.247142,1615.898612,203.530358,4.441984e-16
min,536365.000000,1.000000,2.400000,12370.000000,2.950000,2.893106e+00
25%,545565.000000,4.000000,2.950000,14239.000000,11.800000,2.893106e+00
50%,555002.000000,6.000000,2.950000,15598.000000,17.700000,2.893106e+00
75%,567466.000000,12.000000,2.950000,16931.000000,35.400000,2.893106e+00
max,581538.000000,1930.000000,5.790000,18283.000000,4921.500000,2.893106e+00


#### Adesso abbiamo a disposizione un subset di clienti leali con un alta soglia di spesa e il prezzo medio di vendita per ogni prodotto. Calcolando la distanza tra il prezzo di acquisto in ogni singola transazione e la media del prezzo, ottengo il numero di volte in cui ogni cliente ha acquistato almeno un certo numero di prodotti ad un prezzo scontato (calcolando la percentuale dello sconto rispetto alla media, non potendo conoscere il prezzo di listino) e, ordinando il dataset per tale valore, ottengo la lista dei clienti con un'alta tendenza all'acquisto di stock di prodotti a prezzi scontati.

## 3) Calcolo la percentuale di sconto rispetto alla media per ogni acquisto

In [19]:
cut_df['DiscountPercent'] = 0
cut_df['DiscountPercent'] = cut_df.apply(lambda row: (
    (row['PriceMean'] - row['UnitPrice'])/row['PriceMean'])*100, axis=1)

/Users/leoar/Desktop/DATA_MINING_PROJ/dev/venv/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [20]:
cut_df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Sales,PriceMean,DiscountPercent
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30,2.893106,11.859423
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,3.768403,10.041469
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00,3.816311,27.940892
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,4.001595,15.283783
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,4.054756,16.394478


## 4) Creo un contatore cui assegno 1 se sono stati acquistati almeno 20 unità del prodotto ad un prezzo scontato di almeno il 15% rispetto alla media 

In [21]:
# creo una colonna contatore, che assegna +1 per ogni prodotto di cui
#si è registrato un acquisto di almeno 20 unità con uno sconto di 
#almeno il 15% rispetto al prezzo medio di vendita
cut_df['counter'] = 0
cut_df['counter'] = cut_df.apply(lambda row: 1 if row['DiscountPercent'] >15
                                 and row['Quantity'] > 20 else 0, axis=1)

/Users/leoar/Desktop/DATA_MINING_PROJ/dev/venv/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [22]:
cut_df[cut_df['counter']==1].head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Sales,PriceMean,DiscountPercent,counter
65,536374,21258,VICTORIAN SEWING BOX LARGE,32,2010-12-01 09:09:00,10.95,15100.0,United Kingdom,350.40,12.952358,15.459412,1
82,536376,22114,HOT WATER BOTTLE TEA AND SYMPATHY,48,2010-12-01 09:32:00,3.45,15291.0,United Kingdom,165.60,4.212192,18.094908,1
96,536378,21212,PACK OF 72 RETROSPOT CAKE CASES,120,2010-12-01 09:37:00,0.42,14688.0,United Kingdom,50.40,0.548212,23.387248,1
102,536378,85071B,RED CHARLIE+LOLA PERSONAL DOORSIGN,96,2010-12-01 09:37:00,0.38,14688.0,United Kingdom,36.48,0.622455,38.951366,1
173,536386,84880,WHITE WIRE EGG HOLDER,36,2010-12-01 09:57:00,4.95,16029.0,United Kingdom,178.20,7.061029,29.896907,1


In [23]:
sum_of_disc = cut_df.groupby(['CustomerID'])['counter'].sum()

In [26]:
sum_of_disc.sort_values(ascending=False).head()

CustomerID
14646.0    623
14298.0    604
18102.0    301
13694.0    270
14911.0    270
Name: counter, dtype: int64

## 5) Seleziono i 100 clienti con il più alto valore di counter ed costruisco un dataset in cui ad ogni cliente corrisponde un insieme di valori acquistati e la rispettiva data di acquisto di tale insieme.

In [27]:
max_discounter = sum_of_disc.sort_values(ascending=False).head(100)

In [28]:
max_discounter

CustomerID
14646.0    623
14298.0    604
18102.0    301
13694.0    270
14911.0    270
          ... 
13599.0     18
13225.0     18
16558.0     18
13576.0     18
15125.0     18
Name: counter, Length: 100, dtype: int64

In [29]:
max_disc_df = cut_df[cut_df['CustomerID'].isin(
        list(max_discounter.index))]

In [30]:
max_disc_df = max_disc_df[max_disc_df['counter']==1]

In [31]:
max_disc_df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Sales,PriceMean,DiscountPercent,counter
173,536386,84880,WHITE WIRE EGG HOLDER,36,2010-12-01 09:57:00,4.95,16029.0,United Kingdom,178.20,7.061029,29.896907,1
174,536386,85099C,JUMBO BAG BAROQUE BLACK WHITE,100,2010-12-01 09:57:00,1.65,16029.0,United Kingdom,165.00,2.012231,18.001448,1
175,536386,85099B,JUMBO BAG RED RETROSPOT,100,2010-12-01 09:57:00,1.65,16029.0,United Kingdom,165.00,2.015878,18.149794,1
176,536387,79321,CHILLI LIGHTS,192,2010-12-01 09:58:00,3.82,16029.0,United Kingdom,733.44,5.428760,29.634017,1
177,536387,22780,LIGHT GARLAND BUTTERFILES PINK,192,2010-12-01 09:58:00,3.37,16029.0,United Kingdom,647.04,4.184394,19.462650,1


In [32]:
buyed_items_per_cust = max_disc_df.groupby(
    ['CustomerID','InvoiceDate'])['StockCode'].apply(set)

In [33]:
buyed_items_per_cust

CustomerID  InvoiceDate        
12415.0     2011-01-06 11:12:00    {22077, 22492, 21891, 22522, 22620, 21238, 223...
            2011-02-15 09:52:00    {21212, 23230, 20973, 22617, 23231, 22907, 226...
            2011-03-03 10:59:00    {21509, 22993, 22045, 22028, 21984, 21086, 220...
            2011-05-17 15:42:00    {23173, 23253, 22549, 21900, 85132A, 23298, 22...
            2011-05-20 14:13:00                                              {22712}
                                                         ...                        
18102.0     2011-11-04 10:38:00    {23536, 23528, 22189, 23403, 23322, 22470, 235...
            2011-11-04 10:57:00    {23535, 23524, 23531, 23541, 23529, 23527, 221...
            2011-11-28 12:55:00                                              {85152}
            2011-12-08 18:43:00    {23535, 23531, 23401, 22188, 23530, 82484, 235...
            2011-12-09 11:50:00                                              {23404}
Name: StockCode, Length: 1338, dt

In [34]:
buyed_items_per_cust.to_csv('buyed_items_per_customer.csv')

### Attraverso il dataset creato, dopo aver individuato le regole di associazione, trovando le intersezioni tra l'ultimo acquisto di ogni cliente e gli antecedents di tali regole, possiamo individuare un insieme di prodotti da consigliare a tale cliente per il prossimo acquisto.

### Al fine di creare un sistema più coerente, decido di uitilizzare, per l'analisi di associazione, esclusivamente l'insieme delle transazioni generate dai 100 clienti selezionati. Questo al fine di poter dare un consiglio di acquisto basato non solo sulla frequenza di acquisto dei prodotti ma anche sulle caratteristiche e propensioni di acquisto di clienti simili a quelli presi in considerazione. Costruisco quindi il dataframe contenente tali dati.

In [35]:
best_cust_df = df[df['CustomerID'].isin(
    list(max_discounter.index))]
best_cust_df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Sales
106,536381,22139,RETROSPOT TEA SET CERAMIC 11 PC,23,2010-12-01 09:41:00,4.25,15311.0,United Kingdom,97.75
107,536381,84854,GIRLY PINK TOOL SET,5,2010-12-01 09:41:00,4.95,15311.0,United Kingdom,24.75
108,536381,22411,JUMBO SHOPPER VINTAGE RED PAISLEY,10,2010-12-01 09:41:00,1.95,15311.0,United Kingdom,19.50
109,536381,82567,"AIRLINE LOUNGE,METAL SIGN",2,2010-12-01 09:41:00,2.10,15311.0,United Kingdom,4.20
110,536381,21672,WHITE SPOT RED CERAMIC DRAWER KNOB,6,2010-12-01 09:41:00,1.25,15311.0,United Kingdom,7.50
...,...,...,...,...,...,...,...,...,...
397831,581580,37500,TEA TIME TEAPOT IN GIFT BOX,1,2011-12-09 12:20:00,4.95,12748.0,United Kingdom,4.95
397837,581583,20725,LUNCH BAG RED RETROSPOT,40,2011-12-09 12:23:00,1.45,13777.0,United Kingdom,58.00
397838,581583,85038,6 CHOCOLATE LOVE HEART T-LIGHTS,36,2011-12-09 12:23:00,1.85,13777.0,United Kingdom,66.60
397839,581584,20832,RED FLOCK LOVE HEART PHOTO FRAME,72,2011-12-09 12:25:00,0.72,13777.0,United Kingdom,51.84


In [36]:
best_cust_df.to_csv('best_customers_df.csv', index = False)